# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'alicia'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

80 rows found with non string elements for column comment (0.65%)
Deleting 2326 columns for which max target value is over 7 (18.76%)
9991 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

Usamos el número de tópicos con mejor métrica de coherencia

In [6]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=6, id2word=dictionary, passes=10)

CPU times: user 23.3 s, sys: 22.5 ms, total: 23.3 s
Wall time: 23.4 s


In [7]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=6)
for topic in topics:
    print(topic)

(0, '0.032*"cumplir" + 0.028*"acuerdos" + 0.023*"condiciones" + 0.020*"debe" + 0.014*"contexto" + 0.013*"importante" + 0.012*"si" + 0.012*"plazos" + 0.012*"proyecto" + 0.008*"siempre"')
(1, '0.054*"proyecto" + 0.022*"entregar" + 0.017*"si" + 0.016*"criterios" + 0.015*"priorizar" + 0.014*"usuarios" + 0.014*"importante" + 0.014*"tiempo" + 0.014*"bien" + 0.014*"plazo"')
(2, '0.018*"cumplir" + 0.017*"plazos" + 0.016*"debe" + 0.016*"grupo" + 0.015*"proyecto" + 0.012*"acuerdo" + 0.012*"criterios" + 0.009*"postura" + 0.009*"plazo" + 0.008*"posible"')
(3, '0.030*"Alicia" + 0.028*"contexto" + 0.026*"proyecto" + 0.015*"mundial" + 0.015*"pandemia" + 0.014*"debería" + 0.014*"contingencia" + 0.014*"debe" + 0.012*"acuerdos" + 0.011*"situación"')
(4, '0.024*"trabajo" + 0.006*"va" + 0.006*"solución" + 0.004*"Por" + 0.004*"solo" + 0.004*"opción" + 0.004*"día" + 0.004*"\'" + 0.004*"Ya" + 0.004*"ética"')
(5, '0.032*"reputación" + 0.026*"ser" + 0.022*"si" + 0.022*"transparencia" + 0.018*"usuarios" + 0.014

In [8]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Es importante producir producto aceptable incluso si demora pues hacer producto mal puede llevar repercuciones grandes

Topic 1: 0.9505146741867065


## Correlaciones

In [10]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

In [11]:
from scipy.stats import pearsonr

# Only compute pearson prod-moment correlations between feature
# columns and target column
target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]
print("Feature-Target Correlations")
print(feature_target_corr)

Feature-Target Correlations
{'Topic1_sel': 0.31869272901725754, 'Topic2_sel': -0.1732275803904305, 'Topic3_sel': 0.017360158480722546, 'Topic4_sel': 0.43204209804924637, 'Topic5_sel': 0.08314547653820803, 'Topic6_sel': -0.32136249034407566}
